In [1]:
# LSST stack imports
# from lsst.analysis.ap import apdb
from lsst.daf.butler import Butler

repo = "/repo/main"
collection = "LSSTComCam/runs/DRP/DP1/w_2025_03/DM-48478"
instrument = "LSSTComCam"

# afwDisplay.setDefaultBackend('firefly')
# display1 = afwDisplay.Display(frame=1)
# display1.getClient().get_firefly_url()

butler = Butler(repo, collections=collection)

In [14]:
import astropy.units as u
import lsst.geom as geom
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
from astropy.visualization import ZScaleInterval
from astropy.visualization.wcsaxes import SphericalCircle
from astropy.wcs import WCS
from reproject import reproject_interp


def get_cutout(data_id, ra, dec, size=100):
    wcs = butler.get('calexp.wcs', **data_id)
    detector_box = butler.get('calexp.detector', **data_id).getBBox()
    xy = geom.PointI(wcs.skyToPixel(geom.SpherePoint(ra, dec, geom.degrees)))

    cutout_size = geom.ExtentI(size, size)
    cutout_box = geom.BoxI(xy - cutout_size // 2, cutout_size)
    cutout_box = cutout_box.clippedTo(detector_box)

    return butler.get('calexp', **data_id, parameters={'bbox': cutout_box}), cutout_box


def plot_cutout(visit_id, detector_id, ra, dec, size=100, reproject_wcs=None):
    data_id = dict(visit=visit_id, detector=detector_id, instrument="LSSTComCam")
    
    cutout, cutout_box = get_cutout(data_id, ra, dec, size=size)
    cutout_box_min_corner = cutout_box.getCorners()[0]

    ap10_flux = get_cutout(data_id, ra, dec, size=10)[0].getImage().getArray().sum()

    astropy_wcs = WCS(cutout.wcs.getFitsMetadata())
    astropy_wcs.wcs.crpix[0] -= cutout_box_min_corner.x
    astropy_wcs.wcs.crpix[1] -= cutout_box_min_corner.y

    image_array = cutout.getImage().getArray()
    # cutout.getMask()
    # cutout.getVariance()
    if reproject_wcs is not None:
        image_array, _footprint = reproject_interp(
            (image_array, astropy_wcs),
            reproject_wcs,
            shape_out=(size, size),
            order='bicubic',
        )
        astropy_wcs = reproject_wcs
    
    interval = ZScaleInterval()
    _vmin, vmax = interval.get_limits(image_array)

    plt.subplot(projection=astropy_wcs)
    plt.title(f"visit: {visit_id}, detector: {detector_id}, {ra=:.5f}, {dec=:.5f}\n{ap10_flux=:.0f}")
    lon = plt.gca().coords[0]
    lat = plt.gca().coords[1]
    lon.set_major_formatter('dd:mm:ss')
    lat.set_major_formatter('dd:mm:ss')
    lon.set_ticklabel(exclude_overlapping=False)
    lat.set_ticklabel(exclude_overlapping=False)
    lon.set_ticklabel_position('b')
    lat.set_ticklabel_position('l')
    plt.imshow(image_array, cmap='gray', vmin=0, vmax=vmax, origin='lower')
    plt.xlabel('RA')
    plt.ylabel('Dec')
    # circle patch for source position
    circle = SphericalCircle(SkyCoord(ra, dec, unit=u.deg), 1*u.arcsec, edgecolor='red', facecolor='none', 
                             transform=plt.gca().get_transform('icrs'))
    plt.gca().add_patch(circle)
    lon.set_ticks(spacing=3. * u.arcsec)
    lat.set_ticks(spacing=3. * u.arcsec)
    plt.grid(color='white', alpha=0.5, ls='solid')
    plt.colorbar()
    plt.show()

In [ ]:
ra= 53.211946
dec= -28.201129
size = 120

def create_wcs(ra, dec, size=100):
    w = WCS(naxis=2)
    w.wcs.crpix = [size / 2, size / 2]
    w.wcs.crval = [ra, dec]
    w.wcs.cdelt = [0.2 / 3600] * 2  # LSST pixel scale
    w.wcs.ctype = ["RA---TAN", "DEC--TAN"]  # Use TAN projection
    return w

reproject_wcs = create_wcs(ra, dec, size=size)

plot_cutout(
    visit_id = 2024110900195,
    detector_id = 3,
    ra=ra,
    dec=dec,
    reproject_wcs=reproject_wcs,
    size=size,
)
plot_cutout(
    visit_id = 2024120100184,
    detector_id = 8,
    ra=ra,
    dec=dec,
    reproject_wcs=reproject_wcs,
    size=size,
)